In [ ]:
# import tensorflow as tf
# from tensorflow.examples.tutorials.mnist import input_data
# # from tensorflow.keras.datasets import mnist
# mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# learning_rate = 1e-4
# keep_prob_rate = 0.7 # 
# max_epoch = 2000
# def compute_accuracy(v_xs, v_ys):
#     global prediction
#     y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1})
#     correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
#     accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#     result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})
#     return result

# def weight_variable(shape):
#     initial = tf.truncated_normal(shape, stddev=0.1)
#     return tf.Variable(initial)

# def bias_variable(shape):
#     initial = tf.constant(0.1, shape=shape)
#     return tf.Variable(initial)

# def conv2d(x, W):
#     # 每一维度  滑动步长全部是 1， padding 方式 选择 same
#     # 提示 使用函数  tf.nn.conv2d
    
#     return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

# def max_pool_2x2(x):
#     # 滑动步长 是 2步; 池化窗口的尺度 高和宽度都是2; padding 方式 请选择 same
#     # 提示 使用函数  tf.nn.max_pool
    
#     return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') 

# # define placeholder for inputs to network
# xs = tf.placeholder(tf.float32, [None, 784])/255.
# ys = tf.placeholder(tf.float32, [None, 10])
# keep_prob = tf.placeholder(tf.float32)
# x_image = tf.reshape(xs, [-1, 28, 28, 1])

# #  卷积层 1
# ## conv1 layer ##

# W_conv1 = weight_variable([7, 7, 1, 32])   # patch 7x7, in size 1, out size 32
# b_conv1 = bias_variable([32])
# h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)  # 卷积  自己选择 选择激活函数
# h_pool1 = max_pool_2x2(h_conv1)  # 池化               

# # 卷积层 2
# W_conv2 = weight_variable([5, 5, 32, 64])   # patch 5x5, in size 32, out size 64
# b_conv2 = bias_variable([64])
# h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)    # 卷积  自己选择 选择激活函数
# h_pool2 = max_pool_2x2(h_conv2)  # 池化

# #  全连接层 1
# ## fc1 layer ##
# W_fc1 = weight_variable([7*7*64, 1024])
# b_fc1 = bias_variable([1024])

# h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
# h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
# h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# # 全连接层 2
# ## fc2 layer ##
# W_fc2 = weight_variable([1024, 10])
# b_fc2 = bias_variable([10])
# prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)


# # 交叉熵函数
# cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),
#                                               reduction_indices=[1]))
# train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

# with tf.Session() as sess:
#     init = tf.global_variables_initializer()
#     sess.run(init)
    
#     for i in range(max_epoch):
#         batch_xs, batch_ys = mnist.train.next_batch(100)
#         sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob:keep_prob_rate})
#         if i % 100 == 0:
#             print(compute_accuracy(
#                 mnist.test.images[:1000], mnist.test.labels[:1000]))

import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist

# 超参数设置
learning_rate = 1e-4
keep_prob_rate = 0.7  # Dropout 比例
max_epoch = 3
BATCH_SIZE = 50

# 加载 MNIST 数据集
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 数据预处理：归一化，并调整输入形状
x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
x_test = x_test.reshape(-1, 28, 28, 1) / 255.0

# 将标签转换为 one-hot 编码
y_train_one_hot = tf.keras.utils.to_categorical(y_train, 10)
y_test_one_hot = tf.keras.utils.to_categorical(y_test, 10)

# 定义卷积神经网络模型
class CNN(tf.keras.Model):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(32, kernel_size=7, activation='relu', padding='same', input_shape=(28, 28, 1))
        self.pool1 = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2, padding='same')
        self.conv2 = tf.keras.layers.Conv2D(64, kernel_size=5, activation='relu', padding='same')
        self.pool2 = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2, padding='same')
        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(1024, activation='relu')
        self.dropout = tf.keras.layers.Dropout(keep_prob_rate)
        self.fc2 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# 定义准确率计算函数
def compute_accuracy(v_xs, v_ys, model):
    predictions = model(v_xs, training=False)  # 预测
    correct_prediction = np.argmax(predictions, axis=1) == np.argmax(v_ys, axis=1)  # 确保 v_ys 是 one-hot 编码
    accuracy = np.mean(correct_prediction)
    return accuracy

# 定义训练函数
def train(model):
    optimizer = tf.keras.optimizers.Adam(learning_rate)
    loss_func = tf.keras.losses.CategoricalCrossentropy()
    
    for epoch in range(max_epoch):
        for step in range(0, len(x_train), BATCH_SIZE):
            x_batch = x_train[step:step+BATCH_SIZE]
            y_batch = y_train_one_hot[step:step+BATCH_SIZE]
            
            with tf.GradientTape() as tape:
                output = model(x_batch, training=True)  # 训练模式下
                loss = loss_func(y_batch, output)
            
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
            
            if step % 100 == 0:
                accuracy = compute_accuracy(x_test[:1000], y_test_one_hot[:1000], model)
                print(f"Epoch {epoch}, Step {step}, Test accuracy: {accuracy}")

# 训练模型
if __name__ == '__main__':
    model = CNN()
    train(model)




Epoch 0, Step 0, Test accuracy: 0.139
Epoch 0, Step 100, Test accuracy: 0.288
Epoch 0, Step 200, Test accuracy: 0.278
Epoch 0, Step 300, Test accuracy: 0.339
Epoch 0, Step 400, Test accuracy: 0.421
Epoch 0, Step 500, Test accuracy: 0.429
Epoch 0, Step 600, Test accuracy: 0.426
Epoch 0, Step 700, Test accuracy: 0.42
Epoch 0, Step 800, Test accuracy: 0.47
Epoch 0, Step 900, Test accuracy: 0.534
Epoch 0, Step 1000, Test accuracy: 0.567
Epoch 0, Step 1100, Test accuracy: 0.548
Epoch 0, Step 1200, Test accuracy: 0.545
Epoch 0, Step 1300, Test accuracy: 0.575
Epoch 0, Step 1400, Test accuracy: 0.593
Epoch 0, Step 1500, Test accuracy: 0.629
Epoch 0, Step 1600, Test accuracy: 0.671
Epoch 0, Step 1700, Test accuracy: 0.673
Epoch 0, Step 1800, Test accuracy: 0.676
Epoch 0, Step 1900, Test accuracy: 0.671
Epoch 0, Step 2000, Test accuracy: 0.672
Epoch 0, Step 2100, Test accuracy: 0.681
Epoch 0, Step 2200, Test accuracy: 0.694
Epoch 0, Step 2300, Test accuracy: 0.689
Epoch 0, Step 2400, Test accur